In [1]:
# train_resnet_classifier.py
import os
import time
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, MaxPool1D, Dense, 
                                   Dropout, BatchNormalization, ReLU,
                                   GlobalAveragePooling1D, Add, Reshape)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, 
                                      ModelCheckpoint)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# ---------- GPU配置 ----------
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- ResNet块定义 ----------
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, name=None):
    """残差块实现（简化版）"""
    shortcut = x
    if conv_shortcut:
        shortcut = Conv1D(filters, 1, strides=stride, name=name+'_shortcut')(x)
        shortcut = BatchNormalization(name=name+'_bn_shortcut')(shortcut)
    
    x = Conv1D(filters, kernel_size, strides=stride, padding='same', name=name+'_conv1')(x)
    x = BatchNormalization(name=name+'_bn1')(x)
    x = ReLU(name=name+'_relu1')(x)
    
    x = Conv1D(filters, kernel_size, padding='same', name=name+'_conv2')(x)
    x = BatchNormalization(name=name+'_bn2')(x)
    
    x = Add(name=name+'_add')([shortcut, x])
    x = ReLU(name=name+'_out')(x)
    return x

# ---------- ResNet分类模型 ----------
def build_resnet(input_shape, num_classes):
    """构建纯分类的ResNet模型"""
    inputs = Input(shape=input_shape)
    
    # 重塑输入以增加特征维度
    x = Reshape((input_shape[0], 1))(inputs)
    
    # 初始卷积层
    x = Conv1D(64, 7, strides=2, padding='same', name='conv1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = ReLU(name='relu1')(x)
    x = MaxPool1D(3, strides=2, padding='same', name='pool1')(x)
    
    # 残差块堆叠
    x = residual_block(x, 64, name='resblock1_1')
    x = residual_block(x, 64, name='resblock1_2')
    
    x = residual_block(x, 128, stride=2, conv_shortcut=True, name='resblock2_1')
    x = residual_block(x, 128, name='resblock2_2')
    
    x = residual_block(x, 256, stride=2, conv_shortcut=True, name='resblock3_1')
    x = residual_block(x, 256, name='resblock3_2')
    
    # 全局平均池化
    x = GlobalAveragePooling1D(name='global_avg_pool')(x)
    
    # 分类头
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    outputs = Dense(num_classes, activation='softmax', name='classification')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(1e-3),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    return model

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载数据集并处理键名差异"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
    else:
        type2label = {
            "satellite_signal": 0,
            "single_tone": 1,
            "comb_spectra": 2,
            "sweeping": 3,
            "pulse": 4,
            "frequency_hopping": 5,
            "noise_fm": 6,
            "noise_am": 7,
            "random_combination": 8
        }
    
    label2name = {int(i): str(type_names[k]) for k, i in type2label.items()}
    
    dataset.update({
        "type2label": type2label,
        "label2name": label2name
    })
    return dataset

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据（仅分类任务）"""
    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]])
    
    # 数据集分割（保持分层抽样）
    X_train, X_test, y_train, y_test = train_test_split(
        signals, dataset["labels"],
        test_size=0.3, random_state=42, 
        stratify=dataset["labels"]
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test,
        test_size=0.5, random_state=42,
        stratify=y_test
    )
    
    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_train": y_train, "y_val": y_val, "y_test": y_test,
        "label2name": dataset["label2name"],
        "L": dataset["L"]
    }

# ---------- 数据增强 ----------
@tf.function
def aug_fn(x):
    """信号增强函数"""
    x = tf.cast(x, tf.float32)
    if tf.random.uniform([]) > 0.2:
        snr = tf.random.uniform([], 5., 25.)
        noise = tf.random.normal(tf.shape(x)) * tf.math.reduce_std(x) * 10.0**(-snr/20.0)
        x = x + noise
    if tf.random.uniform([]) > 0.3:
        shift = tf.random.uniform([], -100, 100, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=0)
    if tf.random.uniform([]) > 0.3:
        scale = tf.random.uniform([], 0.7, 1.3)
        x = x * scale
    return x

# ---------- 训练单个模型 ----------
def train_single_model(data, model_idx=0, epochs=120, batch=128):
    """训练单个ResNet分类模型"""
    model = build_resnet((data["L"],), len(data["label2name"]))
    
    # 类别权重（处理不平衡）
    cls_weights = compute_class_weight('balanced', 
                                     classes=np.unique(data['y_train']), 
                                     y=data['y_train'])
    sample_weights = np.array([cls_weights[lab] for lab in data['y_train']])
    
    # 数据管道
    train_ds = tf.data.Dataset.from_tensor_slices(
        (data['X_train'], data['y_train'], sample_weights)
    )
    train_ds = train_ds.map(lambda x, y, w: (aug_fn(x), y, w))
    train_ds = train_ds.shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE)
    
    val_ds = tf.data.Dataset.from_tensor_slices(
        (data['X_val'], data['y_val'])
    ).batch(batch).prefetch(tf.data.AUTOTUNE)
    
    # 回调函数
    os.makedirs("models", exist_ok=True)
    ckpt = f"models/resnet_classifier_{model_idx}.keras"
    callbacks = [
        EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=20, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=10),
        ModelCheckpoint(ckpt, save_best_only=True, monitor='val_sparse_categorical_accuracy')
    ]
    
    print(f"\n🔥 训练ResNet模型 {model_idx + 1}...")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    return model

# ---------- 主函数 ----------
def main():
    os.makedirs("models", exist_ok=True)
    os.makedirs("visualizations", exist_ok=True)
    
    print("=" * 80)
    print("🚀 开始训练ResNet干扰分类模型")
    print("=" * 80)

    # 加载数据
    print("⏳ 加载数据集...")
    dataset = load_dataset("/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz")
    data = preprocess_data(dataset)
    
    # 训练参数
    n_models = 3
    epochs = 120
    batch_size = 128

    # 训练模型
    model = train_single_model(data, model_idx=0, epochs=epochs, batch=batch_size)
    
    # 评估测试集
    test_loss, test_acc = model.evaluate(data['X_test'], data['y_test'], verbose=0)
    print(f"\n📊 测试集准确率: {test_acc:.4f}")
    
    # 保存最终模型
    model.save("models/resnet_classifier_final.keras")
    print("\n✅ 训练完成！模型已保存至 models/resnet_classifier_final.keras")

if __name__ == "__main__":
    main()

2025-10-21 18:06:37.355519: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-21 18:06:37.394079: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-21 18:06:37.924485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 开始训练ResNet干扰分类模型
⏳ 加载数据集...


2025-10-21 18:06:57.551110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46590 MB memory:  -> device: 0, name: NVIDIA vGPU-48GB, pci bus id: 0000:d8:00.0, compute capability: 8.9



🔥 训练ResNet模型 1...
Epoch 1/120


2025-10-21 18:06:58.774609: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [56700]
	 [[{{node Placeholder/_1}}]]
2025-10-21 18:06:58.774824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [56700]
	 [[{{node Placeholder/_1}}]]
2025-10-21 18:07:03.435966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-10-21 18:07:03.689309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-10-21 

443/443 [==============================] - ETA: 0s - loss: 1.0382 - sparse_categorical_accuracy: 0.5909

2025-10-21 18:07:21.247813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [12150]
	 [[{{node Placeholder/_1}}]]


443/443 [==============================] - 24s 31ms/step - loss: 1.0382 - sparse_categorical_accuracy: 0.5909 - val_loss: 0.8729 - val_sparse_categorical_accuracy: 0.6290 - lr: 0.0010
Epoch 2/120
443/443 [==============================] - 14s 30ms/step - loss: 0.8388 - sparse_categorical_accuracy: 0.6524 - val_loss: 0.8267 - val_sparse_categorical_accuracy: 0.6600 - lr: 0.0010
Epoch 3/120
443/443 [==============================] - 14s 30ms/step - loss: 0.7816 - sparse_categorical_accuracy: 0.6692 - val_loss: 0.7220 - val_sparse_categorical_accuracy: 0.6930 - lr: 0.0010
Epoch 4/120
443/443 [==============================] - 13s 28ms/step - loss: 0.7551 - sparse_categorical_accuracy: 0.6797 - val_loss: 1.4538 - val_sparse_categorical_accuracy: 0.5230 - lr: 0.0010
Epoch 5/120
443/443 [==============================] - 13s 28ms/step - loss: 0.7348 - sparse_categorical_accuracy: 0.6864 - val_loss: 1.1451 - val_sparse_categorical_accuracy: 0.5851 - lr: 0.0010
Epoch 6/120
443/443 [===========

In [2]:
# evaluate_resnet_classifier.py
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                           classification_report)
from sklearn.preprocessing import StandardScaler
import matplotlib.font_manager as fm

# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载数据集并处理键名差异"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
        print("⚠️ 未找到干扰类型名称，使用默认值")
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
    else:
        type2label = {
            "satellite_signal": 0,
            "single_tone": 1,
            "comb_spectra": 2,
            "sweeping": 3,
            "pulse": 4,
            "frequency_hopping": 5,
            "noise_fm": 6,
            "noise_am": 7,
            "random_combination": 8
        }
    
    label2name = {int(i): str(type_names[k]) for k, i in type2label.items()}
    
    return {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "label2name": label2name,
        "L": int(data["L"])
    }

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据"""
    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]])
    
    return signals, dataset["labels"], dataset["jnr_values"], dataset["label2name"]

# ---------- 加载模型 ----------
def load_model(model_path):
    """加载训练好的ResNet模型"""
    if os.path.exists(model_path):
        try:
            model = tf.keras.models.load_model(model_path)
            print(f"✅ 成功加载模型: {model_path}")
            return model
        except Exception as e:
            print(f"❌ 加载模型失败: {e}")
            return None
    print(f"⚠️ 未找到模型: {model_path}")
    return None

# ---------- 计算JNR准确率 ----------
def calculate_jnr_accuracy(y_true, y_pred, jnr_values, jnr_range):
    """计算每个JNR下的分类准确率"""
    jnr_acc = {}
    for jnr in jnr_range:
        mask = jnr_values == jnr
        if np.sum(mask) > 0:
            jnr_acc[jnr] = accuracy_score(y_true[mask], y_pred[mask])
        else:
            jnr_acc[jnr] = np.nan
    return jnr_acc

# ---------------------- 绘制混淆矩阵 ----------------------
def plot_confusion_matrix(cm, labels, title, xlabel, ylabel, filename, dpi=150, rotate_x=False):
    """专业混淆矩阵可视化"""
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_normalized = np.nan_to_num(cm_normalized)

    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(cm_normalized,
                     annot=True,
                     fmt='.2f',
                     cmap='Blues',
                     xticklabels=labels,
                     yticklabels=labels,
                     square=True,
                     annot_kws={"size": 14})
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)

    plt.title(title, pad=20, fontsize=18)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(rotation=45 if rotate_x else 0, ha='right' if rotate_x else 'center', fontsize=14)
    plt.yticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi)
    plt.close()

# ---------- 绘制JNR准确率曲线 ----------
def plot_jnr_accuracy(jnr_acc, filename):
    """绘制JNR-准确率曲线"""
    plt.figure(figsize=(10, 6))
    valid_jnr = [j for j in jnr_acc if not np.isnan(jnr_acc[j])]
    valid_acc = [jnr_acc[j] for j in valid_jnr]
    
    plt.plot(valid_jnr, valid_acc, 'o-', linewidth=2, markersize=8)
    plt.xlabel('JNR (dB)', fontsize=12)
    plt.ylabel('分类准确率', fontsize=12)
    plt.title('不同JNR下的干扰识别准确率', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(list(jnr_acc.keys()))
    plt.ylim(0, 1.05)
    plt.savefig(filename, dpi=300)
    plt.close()

# ---------- 主评估函数 ----------
def evaluate(model_path="models/resnet_classifier_final.keras", 
             npz_path="/path/to/dataset.npz"):
    """主评估流程"""
    os.makedirs("visualizations", exist_ok=True)
    os.makedirs("reports", exist_ok=True)
    
    print("=" * 80)
    print("🚀 ResNet干扰分类模型评估")
    print("=" * 80)

    # 1. 加载数据
    print("⏳ 加载数据集...")
    dataset = load_dataset(npz_path)
    X_test, y_test, jnr_test, label2name = preprocess_data(dataset)
    
    # 2. 加载模型
    print("⏳ 加载模型...")
    model = load_model(model_path)
    if not model:
        print("❌ 评估终止：无法加载模型")
        return
    
    # 3. 预测
    print("⏳ 进行预测...")
    y_pred = np.argmax(model.predict(X_test, verbose=1), axis=1)
    
    # 4. 计算JNR准确率
    print("⏳ 计算各JNR下的准确率...")
    jnr_range = np.arange(-10, 31, 5)  # [-10, -5, 0, 5, ..., 30]
    jnr_acc = calculate_jnr_accuracy(y_test, y_pred, jnr_test, jnr_range)
    
    # 5. 生成混淆矩阵
    print("⏳ 生成混淆矩阵...")
    cm = confusion_matrix(y_test, y_pred)
    class_names = [label2name[i] for i in sorted(label2name.keys())]
    plot_confusion_matrix(
        cm=cm,
        labels=class_names,
        title="ResNet Classfication Confusion Matrix",
        xlabel="Predicted",
        ylabel="True",
        filename="visualizations/ResNet_confusion_matrix.png",
        dpi=300,
        rotate_x=True
    )
    
    # 6. 绘制JNR准确率曲线
    print("⏳ 生成JNR准确率曲线...")
    plot_jnr_accuracy(jnr_acc, "visualizations/resnet_jnr_accuracy.png")
    
    # 7. 保存评估报告
    print("⏳ 生成评估报告...")
    report = {
        "overall_accuracy": float(accuracy_score(y_test, y_pred)),
        "jnr_accuracy": {int(j): float(acc) if not np.isnan(acc) else None 
                         for j, acc in jnr_acc.items()},
        "confusion_matrix": cm.tolist(),
        "class_names": class_names,
        "classification_report": classification_report(
            y_test, y_pred, 
            target_names=class_names,
            output_dict=True
        )
    }
    
    with open("reports/resnet_evaluation_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4, ensure_ascii=False)
    
    # 8. 打印结果
    print("\n" + "="*50)
    print("📊 评估结果摘要")
    print("="*50)
    print(f"整体分类准确率: {report['overall_accuracy']:.4f}")
    
    print("\n各JNR下的分类准确率:")
    for jnr in sorted(jnr_acc.keys()):
        acc = jnr_acc[jnr]
        acc_str = f"{acc:.4f}" if not np.isnan(acc) else "N/A"
        print(f"  JNR={jnr}dB: {acc_str}")
    
    print("\n✅ 评估完成！结果已保存至 reports/resnet_evaluation_report.json")

# ---------- 主程序 ----------
if __name__ == "__main__":
    evaluate(
        model_path="models/resnet_classifier_final.keras",
        npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"
    )

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 ResNet干扰分类模型评估
⏳ 加载数据集...
⏳ 加载模型...
✅ 成功加载模型: models/resnet_classifier_final.keras
⏳ 进行预测...
2532/2532 [==============================] - 13s 5ms/step
⏳ 计算各JNR下的准确率...
⏳ 生成混淆矩阵...
⏳ 生成JNR准确率曲线...
⏳ 生成评估报告...

📊 评估结果摘要
整体分类准确率: 0.8979

各JNR下的分类准确率:
  JNR=-10dB: 0.6414
  JNR=-5dB: 0.7700
  JNR=0dB: 0.8640
  JNR=5dB: 0.9216
  JNR=10dB: 0.9456
  JNR=15dB: 0.9739
  JNR=20dB: 0.9872
  JNR=25dB: 0.9883
  JNR=30dB: 0.9893

✅ 评估完成！结果已保存至 reports/resnet_evaluation_report.json


🚀 ResNet干扰分类模型评估
整体分类准确率: 0.8979

各JNR下的分类准确率:
  JNR=-10dB: 0.6414
  JNR=-5dB: 0.7702
  JNR=0dB: 0.8639
  JNR=5dB: 0.9216
  JNR=10dB: 0.9456
  JNR=15dB: 0.9739
  JNR=20dB: 0.9872
  JNR=25dB: 0.9883
  JNR=30dB: 0.9893